In [14]:
import sys, os

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
%cd /content/drive/MyDrive/
%pwd


os.makedirs ('UClim4HSE', exist_ok = True)



/content/drive/MyDrive


In [17]:
!cd ML4hydromet/
!git clone https://github.com/mvarentsov/UClim4HSE-2025.git

Cloning into 'ML4hydromet-2024'...
remote: Enumerating objects: 152, done.
remote: Counting objects: 100% (152/152), done.
remote: Compressing objects: 100% (133/133), done.
remote: Total 152 (delta 73), reused 60 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (152/152), 30.71 MiB | 10.75 MiB/s, done.
Resolving deltas: 100% (73/73), done.
Updating files: 100% (22/22), done.
